# Crop Sequence Boundary data extraction - states

## import libraries

In [4]:
import fiona
import geopandas as gpd
import shapely  # shapely 2.0
import pyogrio
import pyarrow
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pyproj import CRS
# from mpl_toolkits.basemap import Basemap
from scipy.spatial import KDTree
from shapely.geometry import Point, LineString
import itertools
from operator import itemgetter
from pprint import pprint

In [5]:
# Set GeoPandas to use pyogrio
gpd.options.io_engine = "pyogrio"

In [6]:
gpd.show_versions()


SYSTEM INFO
-----------
python     : 3.12.4 | packaged by conda-forge | (main, Jun 17 2024, 10:13:44) [Clang 16.0.6 ]
executable : /Users/jwhite/miniforge3/envs/siads699b/bin/python
machine    : macOS-14.5-arm64-arm-64bit

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : 3.12.2
GEOS lib   : None
GDAL       : 3.9.1
GDAL data dir: /Users/jwhite/miniforge3/envs/siads699b/share/gdal/
PROJ       : 9.4.0
PROJ data dir: /Users/jwhite/miniforge3/envs/siads699b/share/proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 1.0.1
numpy      : 2.0.0
pandas     : 2.2.2
pyproj     : 3.6.1
shapely    : 2.0.5
pyogrio    : 0.9.0
geoalchemy2: 0.15.2
geopy      : 2.4.1
matplotlib : 3.9.1
mapclassify: 2.6.1
fiona      : 1.9.6
psycopg    : 3.2.1
psycopg2   : 2.9.9 (dt dec pq3 ext lo64)
pyarrow    : 16.1.0


In [7]:
dir(pyogrio)

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__gdal_geos_version__',
 '__gdal_version__',
 '__gdal_version_string__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_compat',
 '_env',
 '_err',
 '_geometry',
 '_io',
 '_ogr',
 '_version',
 '_vsi',
 'core',
 'detect_write_driver',
 'errors',
 'geopandas',
 'get_gdal_config_option',
 'get_gdal_data_path',
 'list_drivers',
 'list_layers',
 'open_arrow',
 'raw',
 'read_arrow',
 'read_bounds',
 'read_dataframe',
 'read_info',
 'set_gdal_config_options',
 'shapely',
 'util',
 'write_arrow',
 'write_dataframe']

## set coordinate reference system for Crop Sequence Bounary

Based on metadata.

In [5]:
# Set the CRS using PROJ string
# custom coordinate reference system for CropSequenceBoundaries
crs_string = CRS.from_proj4("+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs +ellps=GRS80")

## read latest file and extract data

- Latest data is 2016 to 2023

- Restrict analysis to  Colorado, Utah, Arizona, and New Mexico

In [6]:
# csb_filepath = '../data/agricultural/CropSequenceBoundaries/NationalCSB_2016-2023_rev23/CSB1623.gdb/'
csb_filepath = '../datasets/fields/NationalCSB_2016-2023_rev23/CSB1623.gdb/'

In [7]:
layers = fiona.listlayers(csb_filepath)

In [8]:
len(layers)

1

In [9]:
layer = layers[0]
# layer
layer_name = layer
layer_name

'national1623'

In [10]:
# Open the geodatabase layer

n = 3 # number of features to collect = n+1

with fiona.open(csb_filepath, layer=layer_name) as src:
    first_n_features = []
    
    # Iterate over the features in the source
    for i, feature in enumerate(src):
        first_n_features.append(feature)
        
        # Break the loop after collecting 10 features
        if i == n:
            break

In [11]:
first_n_features[:1]

[{'type': 'Feature',
  'id': '1',
  'properties': OrderedDict([('CSBID', '351623000000001'),
               ('CSBYEARS', '1623'),
               ('CSBACRES', 2.777984416614868),
               ('CDL2016', 24),
               ('CDL2017', 24),
               ('CDL2018', 24),
               ('CDL2019', 152),
               ('CDL2020', 152),
               ('CDL2021', 152),
               ('CDL2022', 152),
               ('CDL2023', 152),
               ('STATEFIPS', '35'),
               ('STATEASD', '3530'),
               ('ASD', '30'),
               ('CNTY', 'Union'),
               ('CNTYFIPS', '059'),
               ('INSIDE_X', -650336.707800001),
               ('INSIDE_Y', 1447440.5062),
               ('Shape_Length', 628.3153094395814),
               ('Shape_Area', 11242.14904625621)]),
  'geometry': {'type': 'MultiPolygon',
   'coordinates': [[[(-650269.8805, 1447270.0971000008),
      (-650293.9328000005, 1447271.9411999993),
      (-650317.9850999992, 1447273.7852999996),
 

## features look like...

```

[{'type': 'Feature',
  'id': '1',
  'properties': OrderedDict([('CSBID', '351623000000001'),
               ('CSBYEARS', '1623'),
               ('CSBACRES', 2.777984416614868),
               ('CDL2016', 24),
               ('CDL2017', 24),
               ('CDL2018', 24),
               ('CDL2019', 152),
               ('CDL2020', 152),
               ('CDL2021', 152),
               ('CDL2022', 152),
               ('CDL2023', 152),
               ('STATEFIPS', '35'),
               ('STATEASD', '3530'),
               ('ASD', '30'),
               ('CNTY', 'Union'),
               ('CNTYFIPS', '059'),
               ('INSIDE_X', -650336.707800001),
               ('INSIDE_Y', 1447440.5062),
               ('Shape_Length', 628.3153094395814),
               ('Shape_Area', 11242.14904625621)]),
  'geometry': {'type': 'MultiPolygon',
   'coordinates': [[[(-650269.8805, 1447270.0971000008),
      (-650293.9328000005, 1447271.9411999993),
      (-650317.9850999992, 1447273.7852999996),
      (-650315.6782000009, 1447303.8725000005),
      (-650291.6260000002, 1447302.0284000002),
      (-650286.2434999999, 1447372.2317999993),
      (-650307.9886000007, 1447404.1631000005),
      (-650364.1096000001, 1447408.4662999995),
      (-650361.0335000008, 1447448.5824999996),
      (-650334.6746999994, 1447476.8254000004),
      (-650254.5023999996, 1447470.6784000006),
      (-650269.8805, 1447270.0971000008)]]]}}]


```

In [12]:
# https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt 
#
# colorado FIPS is 08
# Utah FIPS is 49
# Arizona FIPS is 04
# New Mexico FIPS is 35

statefips = [
    4, # Arizona
    8, # Colorado
    35, # New Mexico
    49, # Arizona
]

## test import

In [13]:
import pandas as pd
import geopandas as gpd
from collections import defaultdict
import pyogrio
from shapely.geometry import shape
from pyproj import CRS

# File paths
# csb_filepath = '../data/agricultural/CropSequenceBoundaries/NationalCSB_2016-2023_rev23/CSB1623.gdb/'
csb_filepath = '../datasets/fields/NationalCSB_2016-2023_rev23/CSB1623.gdb/'
# output_dir = '../data/_generated_data/'
output_dir = '../datasets/fields/csb/'

# Define the STATEFIPS values to filter
statefips = ['4', '8', '35', '49']

# Initialize a dictionary to store the features by CDL2023 codes
cdl2023_features = defaultdict(list)
target_count = 3

# Define the custom CRS using the provided PROJ string
crs_string = "+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs +ellps=GRS80"
crs = CRS.from_proj4(crs_string)

# Function to read the file in chunks
def read_chunks(filepath, layer, chunk_size=10000):
    start = 0
    while True:
        chunk = pyogrio.read_dataframe(filepath, layer=layer, skip_features=start, max_features=chunk_size)
        if chunk.empty:
            break
        yield chunk
        start += chunk_size

# Read and process the file in chunks
for chunk in read_chunks(csb_filepath, layer=0):
    # Filter the chunk by STATEFIPS
    chunk_filtered = chunk[chunk['STATEFIPS'].isin(statefips)]
    
    # Process each feature in the chunk
    for _, feature in chunk_filtered.iterrows():
        cdl2023 = feature['CDL2023']
        cdl2023_features[cdl2023].append(feature)
        # Stop collecting if we have reached the target count for each CDL2023 code
        if all(len(features) >= target_count for features in cdl2023_features.values()):
            break
    
    # Check if we have reached the target count for each CDL2023 code
    if all(len(features) >= target_count for features in cdl2023_features.values()):
        break

# Select up to 3 features for each unique CDL2023 code
selected_features = []
for cdl2023, features in cdl2023_features.items():
    selected_features.extend(features[:target_count])

# Convert the collected features to a DataFrame and extract geometries
properties_list = [feature.drop(labels='geometry') for feature in selected_features]
geometries = [shape(feature['geometry']) for feature in selected_features]

# Create a GeoDataFrame from the selected features with the custom CRS
gdf_selected = gpd.GeoDataFrame(properties_list, geometry=geometries, crs=crs)

# Rename columns to avoid truncation issues with Shapefile format
gdf_selected = gdf_selected.rename(columns={
    'Shape_Length': 'Shp_Len',
    'Shape_Area': 'Shp_Area',
    # Add other columns if needed
})

# Print the GeoDataFrame
# print(gdf_selected)

# Optionally, save the GeoDataFrame to a file (e.g., shapefile, GeoJSON)
gdf_selected.to_file(f'{output_dir}selected_features.shp')
gdf_selected.to_file(f'{output_dir}selected_features.geojson', driver='GeoJSON')

In [14]:
# Display the GeoDataFrame in Jupyter Lab
gdf_selected.head()
# gdf_selected

,CSBID,CSBYEARS,CSBACRES,CDL2016,CDL2017,CDL2018,CDL2019,CDL2020,CDL2021,CDL2022,...,STATEFIPS,STATEASD,ASD,CNTY,CNTYFIPS,INSIDE_X,INSIDE_Y,Shp_Len,Shp_Area,geometry
0,351623000000001,1623,2.777984,24,24,24,152,152,152,152,...,35,3530,30,Union,059,-650336.7078,1.447441e+06,628.315309,11242.149046,"MULTIPOLYGON (((-650269.880 1447270.097, -6502..."
5,351623000000006,1623,24.183211,24,24,24,152,152,152,152,...,35,3530,30,Union,059,-650353.4439,1.447107e+06,1761.059250,97866.372287,"MULTIPOLYGON (((-650317.985 1447273.785, -6503..."
18,351623000000019,1623,4.837819,24,24,24,4,4,61,61,...,35,3530,30,Harding,021,-663657.5201,1.445929e+06,630.227129,19578.037389,"MULTIPOLYGON (((-663619.003 1446023.964, -6636..."
1,351623000000002,1623,9.293348,24,24,24,24,24,24,24,...,35,3530,30,Union,059,-648203.5992,1.447142e+06,1042.818836,37608.996229,"MULTIPOLYGON (((-648233.790 1447215.119, -6481..."
2,351623000000003,1623,5.775874,24,24,24,24,24,24,24,...,35,3530,30,Union,059,-647571.4914,1.447064e+06,1164.537470,23374.226788,"MULTIPOLYGON (((-647690.945 1446931.542, -6476..."


In [15]:
gdf_selected.columns

Index(['CSBID', 'CSBYEARS', 'CSBACRES', 'CDL2016', 'CDL2017', 'CDL2018',
       'CDL2019', 'CDL2020', 'CDL2021', 'CDL2022', 'CDL2023', 'STATEFIPS',
       'STATEASD', 'ASD', 'CNTY', 'CNTYFIPS', 'INSIDE_X', 'INSIDE_Y',
       'Shp_Len', 'Shp_Area', 'geometry'],
      dtype='object')

## Extract CSBs for Arizona, New Mexico, Utah, and Colorado

FIPS codes are here:
https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt

### test extractions

### extraction all four states

In [16]:
import pandas as pd
import geopandas as gpd
import pyogrio
from shapely.geometry import shape
from pyproj import CRS
import time

# File paths
csb_filepath = '../datasets/fields/NationalCSB_2016-2023_rev23/CSB1623.gdb/'
output_dir = '../datasets/fields/csb/'

# Define the custom CRS using the provided PROJ string
crs_string = "+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs +ellps=GRS80"
custom_crs = CRS.from_proj4(crs_string)

# Define the STATEFIPS value to filter
statefips = '35'  # New Mexico
# statefips = '04' # Arizona

# Initialize a list to store the features
selected_features = []

# Function to read the file in chunks
def read_chunks(filepath, layer, chunk_size=1000):
    start = 0
    while True:
        chunk = pyogrio.read_dataframe(filepath, layer=layer, skip_features=start, max_features=chunk_size)
        if chunk.empty:
            break
        yield chunk
        start += chunk_size

# Timing the execution
start_time = time.time()
chunk_index = 0

# Get the total number of features in the dataset and print the information for debugging
info = pyogrio.read_info(csb_filepath, layer=0)
print(f"Dataset info:")
pprint(info)

# Use the appropriate key to get the total number of features
total_features = info.get('features') or info.get('feature_count')
if total_features is None:
    raise KeyError("Unable to determine the total number of features in the dataset")

print()
print(f"Total number of features in the dataset: {total_features}")

# Set a limit on the number of chunks to process for testing
max_chunks = 5  # Limit the number of chunks for testing

# Read and process the file in chunks
for chunk in read_chunks(csb_filepath, layer=0, chunk_size=1000):
    chunk_start_time = time.time()
    chunk_index += 1

    # Filter the chunk by STATEFIPS
    chunk_filtered = chunk[chunk['STATEFIPS'] == statefips]

    # Collect the filtered features
    selected_features.extend(chunk_filtered.to_dict('records'))

    chunk_end_time = time.time()
    print(f"Processed chunk {chunk_index:>6} in {chunk_end_time - chunk_start_time:.2f} seconds, total features collected: {len(selected_features):>8}")

    # Break the loop if the maximum number of chunks for testing is reached
    if chunk_index >= max_chunks:
        break

    # Break the loop if we have read all features
    if chunk_index * 1000 >= total_features:
        break

# Convert the collected features to a DataFrame and extract geometries
properties_list = [pd.Series(feature).drop(labels='geometry') for feature in selected_features]
geometries = [shape(feature['geometry']) for feature in selected_features]

# Create a GeoDataFrame from the selected features using the custom CRS
gdf_selected = gpd.GeoDataFrame(properties_list, geometry=geometries, crs=custom_crs)

# Rename columns to avoid truncation issues with Shapefile format
gdf_selected = gdf_selected.rename(columns={
    'Shape_Length': 'Shp_Len',
    'Shape_Area': 'Shp_Area',
    # Add other columns if needed
})

# # Save the GeoDataFrame to a file (e.g., shapefile, GeoJSON, Parquet)
# gdf_selected.to_file(f'{output_dir}selected_features_fips_35.shp')
# gdf_selected.to_file(f'{output_dir}selected_features_fips_35.geojson', driver='GeoJSON')
# gdf_selected.to_parquet(f'{output_dir}selected_features_fips_35.parquet')

# Print the execution time
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Script execution time: {elapsed_time:.2f} seconds")


Dataset info:
{'capabilities': {'fast_set_next_by_index': 1,
                  'fast_spatial_filter': 1,
                  'random_read': 1},
 'crs': 'PROJCS["Albers_Conic_Equal_Area",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS '
        '1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]',
 'dtypes': array(['object', 'object', 'float64', 'int32', 'int32', 'int32', 'int32',
       'int32', 'int32', 'int32', 'int32', 'object', 'object', 'object',
       'object', 'object', 'float64', 'float64', 'float64', 'float64'],
      dtype=object),
 'encoding': 'UTF-8',

In [18]:
import pandas as pd
import geopandas as gpd
import pyogrio
from shapely.geometry import shape
from pyproj import CRS
import time
from pprint import pprint
import psutil
import os
from datetime import datetime

# File paths
csb_filepath = '../datasets/fields/NationalCSB_2016-2023_rev23/CSB1623.gdb/'
output_dir = '../datasets/fields/csb/'

# Define the custom CRS using the provided PROJ string
crs_string = "+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs +ellps=GRS80"
custom_crs = CRS.from_proj4(crs_string)

# Define the STATEFIPS values to filter
statefips = ['04', '08', '35', '49']  # Arizona, Colorado, New Mexico, Utah
# statefips = ['04', '08']  # Arizona, Colorado - to check FIPS

# Initialize a dictionary to store the features by state
state_features = {fips: [] for fips in statefips}

# Function to read the file in chunks, starting at a specific chunk
def read_chunks(filepath, layer, chunk_size, total_features, start_chunk=0):
    start = start_chunk * chunk_size
    while start < total_features:
        remaining_features = total_features - start
        if remaining_features < chunk_size:
            chunk_size = remaining_features
        chunk = pyogrio.read_dataframe(filepath, layer=layer, skip_features=start, max_features=chunk_size)
        if chunk.empty:
            break
        yield chunk
        start += chunk_size

# Function to monitor memory usage
def print_memory_usage():
    process = psutil.Process()
    memory_info = process.memory_info()
    print(f"Memory usage: {memory_info.rss / 1024**2:.2f} MB")

# Timing the execution
start_time = time.time()

# Get the total number of features in the dataset
info = pyogrio.read_info(csb_filepath, layer=0)

# Print the dataset info in a nicely formatted way
print("Dataset info:")
pprint(info)

total_features = info.get('features') or info.get('feature_count')
if total_features is None:
    raise KeyError("Unable to determine the total number of features in the dataset")

print(f"Total number of features in the dataset: {total_features}")

# Set chunk size
chunk_size = 1000

# Set the starting chunk index
# start_chunk = 15092
start_chunk=0
chunk_index = start_chunk

# Read and process the file in chunks
for chunk in read_chunks(csb_filepath, layer=0, chunk_size=chunk_size, total_features=total_features, start_chunk=start_chunk):
    chunk_start_time = time.time()
    chunk_index += 1

    # Filter the chunk by STATEFIPS
    chunk_filtered = chunk[chunk['STATEFIPS'].isin(statefips)]

    # Collect the filtered features by state
    for state_fips in statefips:
        state_features[state_fips].extend(chunk_filtered[chunk_filtered['STATEFIPS'] == state_fips].to_dict('records'))

    chunk_end_time = time.time()
    print(f"Processed chunk {chunk_index:>6}, total features collected: {sum(len(features) for features in state_features.values()):>8}")

    # Print memory usage
    print_memory_usage()

# Get the current timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Convert the collected features to a DataFrame and extract geometries
for state_fips, features in state_features.items():
    properties_list = [pd.Series(feature).drop(labels='geometry') for feature in features]
    geometries = [shape(feature['geometry']) for feature in features]

    # Create a GeoDataFrame from the selected features using the custom CRS
    gdf_selected = gpd.GeoDataFrame(properties_list, geometry=geometries, crs=custom_crs)

    # Rename columns to avoid truncation issues with Shapefile format
    gdf_selected = gdf_selected.rename(columns={
        'Shape_Length': 'Shp_Len',
        'Shape_Area': 'Shp_Area',
        # Add other columns if needed
    })

    # Create a subfolder for .shp files for each state
    shapefile_dir = os.path.join(output_dir, f'{timestamp}_selected_features_fips_{state_fips}_shape/')
    os.makedirs(shapefile_dir, exist_ok=True)

    # Save the GeoDataFrame to a file (e.g., shapefile, GeoJSON, Parquet) with a timestamp
    gdf_selected.to_file(f'{shapefile_dir}selected_features_fips_{state_fips}.shp')
    gdf_selected.to_file(f'{output_dir}{timestamp}_selected_features_fips_{state_fips}.geojson', driver='GeoJSON')
    gdf_selected.to_parquet(f'{output_dir}{timestamp}_selected_features_fips_{state_fips}.parquet')

# Print the execution time
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Script execution time: {elapsed_time:.2f} seconds")

Dataset info:
{'capabilities': {'fast_set_next_by_index': 1,
                  'fast_spatial_filter': 1,
                  'random_read': 1},
 'crs': 'PROJCS["Albers_Conic_Equal_Area",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS '
        '1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]',
 'dtypes': array(['object', 'object', 'float64', 'int32', 'int32', 'int32', 'int32',
       'int32', 'int32', 'int32', 'int32', 'object', 'object', 'object',
       'object', 'object', 'float64', 'float64', 'float64', 'float64'],
      dtype=object),
 'encoding': 'UTF-8',

In [19]:
gdf_selected.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 85640 entries, 0 to 85639
Data columns (total 21 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   CSBID      85640 non-null  object  
 1   CSBYEARS   85640 non-null  object  
 2   CSBACRES   85640 non-null  float64 
 3   CDL2016    85640 non-null  int64   
 4   CDL2017    85640 non-null  int64   
 5   CDL2018    85640 non-null  int64   
 6   CDL2019    85640 non-null  int64   
 7   CDL2020    85640 non-null  int64   
 8   CDL2021    85640 non-null  int64   
 9   CDL2022    85640 non-null  int64   
 10  CDL2023    85640 non-null  int64   
 11  STATEFIPS  85640 non-null  object  
 12  STATEASD   85640 non-null  object  
 13  ASD        85640 non-null  object  
 14  CNTY       85640 non-null  object  
 15  CNTYFIPS   85640 non-null  object  
 16  INSIDE_X   85640 non-null  float64 
 17  INSIDE_Y   85640 non-null  float64 
 18  Shp_Len    85640 non-null  float64 
 19  Shp_Area   85640 